Test New CPG (Simple)

In [14]:
import os
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv, GATConv, global_mean_pool, global_add_pool
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import json
import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Utility function to parse and assign node types
def assign_node_type_cpg(node):
    # Example node type detection logic for CPG (AST, CFG, DFG components)
    if 'if' in node or 'while' in node or 'for' in node:
        return 'control_flow'
    elif '=' in node:
        return 'data_flow'
    elif node.startswith('def '):
        return 'function_definition'
    elif '(' in node and ')' in node:
        return 'function_call'
    # Add more types as needed for CPG
    return 'statement'  # Default type

def assign_edge_type_cpg(edge):
    # Check if edge is a string (single line) or tuple (pair of lines)
    if isinstance(edge, str):
        # If it's a single line, treat it as the source line
        source_line = edge
    elif isinstance(edge, tuple) and len(edge) == 2:
        # If it's a tuple of two lines, use the first as the source line
        source_line = edge[0]
    else:
        # If it's neither, return a default edge type
        return 'AST'

    # Control flow edges: connect nodes based on CFG
    if any(keyword in source_line for keyword in ['if', 'for', 'while']):
        return 'control_flow'
    # Data flow edges: assign based on variable assignments, etc.
    if '=' in source_line:
        return 'data_flow'
    return 'AST'  # Default to AST edge type if no other type matches

# Function to generate CPG from code
def generate_cpg(code):
    G = nx.MultiDiGraph()
    lines = code.splitlines()
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue
        node_type = assign_node_type_cpg(line)
        if node_type != 'meaningless':
            G.add_node(i, label=line, type=node_type)
            if i > 0:
                prev_line = lines[i-1].strip()
                edge_type = assign_edge_type_cpg((prev_line, line))
                G.add_edge(i-1, i, type=edge_type)

    isolated_nodes = [node for node, degree in G.degree if degree == 0]
    G.remove_nodes_from(isolated_nodes)
    
    if G.number_of_nodes() == 0:
        return None
    
    mapping = {node: idx for idx, node in enumerate(G.nodes())}
    return nx.relabel_nodes(G, mapping)

# Convert NetworkX graph to PyTorch Geometric Data object
def convert_nx_to_torch_data_cpg(G):
    node_features = torch.tensor(
        [[int(node_data['type'] == t) for t in ['control_flow', 'data_flow', 'AST', 'statement']] 
        for _, node_data in G.nodes(data=True)],
        dtype=torch.float
    )
    
    # Create edge_index correctly
    edge_list = list(G.edges())
    if edge_list:
        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    else:
        # If there are no edges, create an empty edge_index tensor
        edge_index = torch.empty((2, 0), dtype=torch.long)
    
    return Data(x=node_features, edge_index=edge_index)
# Function to load CPG pairs from a dataset
def create_cpg_pairs(data_file, jsonl_file):
    try:
        with open(jsonl_file, 'r') as f:
            url_to_code = {entry['idx']: entry['func'] for entry in (json.loads(line.strip()) for line in f)}
    except FileNotFoundError:
        print(f"Error: File {jsonl_file} not found.")
        return [], []
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in file {jsonl_file}.")
        return [], []

    cpg_pairs, true_labels = [], []
    try:
        with open(data_file, 'r') as f:
            for line in tqdm(f, desc="Creating PDG pairs"):
                try:
                    url1, url2, label = line.strip().split('\t')
                except ValueError:
                    print(f"Warning: Skipping invalid line in {data_file}: {line.strip()}")
                    continue

                if url1 not in url_to_code or url2 not in url_to_code:
                    continue

                try:
                    cpg1, cpg2 = generate_cpg(url_to_code[url1]), generate_cpg(url_to_code[url2])
                    if cpg1 is None or cpg2 is None:
                        continue

                    cpg_pairs.append((convert_nx_to_torch_data_cpg(cpg1), convert_nx_to_torch_data_cpg(cpg2)))
                    true_labels.append(int(label) if label in {'1', '2', '3'} else 0)
                except Exception as e:
                    print(f"Error processing pair {url1} and {url2}: {str(e)}")
                    continue

    except FileNotFoundError:
        print(f"Error: File {data_file} not found.")
        return [], []

    print(f"Created {len(cpg_pairs)} pairs from {data_file}")
    return cpg_pairs, true_labels

class EnhancedGNNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EnhancedGNNLayer, self).__init__()
        self.gin_conv = GINConv(nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, output_dim)
        ))
        self.gat_conv = GATConv(input_dim, output_dim, heads=4, concat=False)
        
    def forward(self, x, edge_index):
        x1 = self.gin_conv(x, edge_index)
        x2 = self.gat_conv(x, edge_index)
        return x1 + x2

class EnhancedGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout_rate=0.3):
        super(EnhancedGNN, self).__init__()
        self.conv_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropout = nn.Dropout(dropout_rate)

        self.conv_layers.append(EnhancedGNNLayer(input_dim, hidden_dim))
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        for _ in range(num_layers - 2):
            self.conv_layers.append(EnhancedGNNLayer(hidden_dim, hidden_dim))
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        self.conv_layers.append(EnhancedGNNLayer(hidden_dim, output_dim))
        self.batch_norms.append(nn.BatchNorm1d(output_dim))

    def forward(self, x, edge_index):
        for conv, bn in zip(self.conv_layers, self.batch_norms):
            x = conv(x, edge_index)
            x = bn(x)
            x = F.relu(x)
            x = self.dropout(x)
        return x

class EnhancedGraphMatcher(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, gembd_vec_dim=128):
        super(EnhancedGraphMatcher, self).__init__()
        self.gnn = EnhancedGNN(input_dim, hidden_dim, output_dim)
        self.mlp = nn.Sequential(
            nn.Linear(output_dim * 4, gembd_vec_dim),
            nn.ReLU(),
            nn.Linear(gembd_vec_dim, gembd_vec_dim),
            nn.ReLU(),
            nn.Linear(gembd_vec_dim, gembd_vec_dim // 2)
        )

    def forward(self, data1, data2):
        h1 = self.gnn(data1.x, data1.edge_index)
        h2 = self.gnn(data2.x, data2.edge_index)

        h1_mean = global_mean_pool(h1, data1.batch if hasattr(data1, 'batch') else torch.zeros(h1.size(0)).long().to(device))
        h2_mean = global_mean_pool(h2, data2.batch if hasattr(data2, 'batch') else torch.zeros(h2.size(0)).long().to(device))
        h1_sum = global_add_pool(h1, data1.batch if hasattr(data1, 'batch') else torch.zeros(h1.size(0)).long().to(device))
        h2_sum = global_add_pool(h2, data2.batch if hasattr(data2, 'batch') else torch.zeros(h2.size(0)).long().to(device))

        h_combined = torch.cat((h1_mean, h2_mean, h1_sum, h2_sum), dim=1)
        return self.mlp(h_combined)

class EnhancedCloneDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, gembd_vec_dim=128):
        super(EnhancedCloneDetector, self).__init__()
        self.graph_matcher = EnhancedGraphMatcher(input_dim, hidden_dim, output_dim, gembd_vec_dim)
        self.classifier = nn.Sequential(
            nn.Linear(gembd_vec_dim // 2, gembd_vec_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(gembd_vec_dim // 4, 4)
        )

    def forward(self, data1, data2):
        out = self.graph_matcher(data1, data2)
        logits = self.classifier(out)
        probabilities = F.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1)
        return logits, probabilities, predicted_class

# Save checkpoint function
def save_checkpoint(model, optimizer, epoch, f1_score, accuracy, precision, recall, metrics, output_dir):
    checkpoint_path = os.path.join(output_dir, f"model_epoch_{epoch}_f1_{f1_score:.4f}.pth")
    
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': {
            'best_f1': f1_score,
            'best_accuracy': accuracy,
            'best_precision': precision,
            'best_recall': recall,
            'per_type_metrics': metrics  
        }
    }, checkpoint_path)
    
    print(f"Checkpoint saved to {checkpoint_path}")

def train(model, train_loader, valid_loader, epochs=100, output_dir="checkpoints"):
    optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.CrossEntropyLoss()
    best_f1 = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            graph_pair, labels = batch
            data1, data2 = graph_pair
            optimizer.zero_grad()

            logits, _, _ = model(data1.to(device), data2.to(device))
            loss = criterion(logits, labels.to(device))
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

        scheduler.step()

        val_loss, val_metrics = evaluate(model, valid_loader)
        val_f1 = val_metrics['overall']['f1']
        val_accuracy = val_metrics['overall']['accuracy']
        val_precision = val_metrics['overall']['precision']
        val_recall = val_metrics['overall']['recall']
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}')
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            save_checkpoint(model, optimizer, epoch, best_f1, val_accuracy, 
                val_precision, val_recall, val_metrics, output_dir)
            
# Evaluation function
def evaluate(model, data_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            graph_pair, labels = batch
            data1, data2 = graph_pair
            logits, _, preds = model(data1.to(device), data2.to(device))
            loss = criterion(logits, labels.to(device))
            total_loss += loss.item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds, all_labels = np.array(all_preds), np.array(all_labels)

    metrics = {'overall': {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='weighted', zero_division=0),
        'f1': f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    }}

    for clone_type in range(4):
        type_preds, type_labels = (all_preds == clone_type), (all_labels == clone_type)
        metrics[f'Type-{clone_type+1}'] = {
            'accuracy': accuracy_score(type_labels, type_preds),
            'precision': precision_score(type_labels, type_preds, zero_division=0),
            'recall': recall_score(type_labels, type_preds, zero_division=0),
            'f1': f1_score(type_labels, type_preds, zero_division=0)
        }

    return total_loss / len(data_loader), metrics

# Main execution logic
input_dim = 4
hidden_dim = 128
output_dim = 256
num_layers = 4
dropout_rate = 0.3
model = EnhancedCloneDetector(input_dim, hidden_dim, output_dim).to(device)

train_file, valid_file = 'test_set/train_5000.txt', 'test_set/train_5000.txt'
jsonl_file = 'test_set/data.jsonl'

train_pdg_pairs, train_labels = create_cpg_pairs(train_file, jsonl_file)
valid_pdg_pairs, valid_labels = create_cpg_pairs(valid_file, jsonl_file)

train_loader = DataLoader(list(zip(train_pdg_pairs, train_labels)), batch_size=64, shuffle=True)
valid_loader = DataLoader(list(zip(valid_pdg_pairs, valid_labels)), batch_size=64)

output_dir = "checkpoints_100_epoch/checkpoints_CPG/5000_sample"
os.makedirs(output_dir, exist_ok=True)

train(model, train_loader, valid_loader, epochs=100, output_dir=output_dir)

Creating PDG pairs: 262it [00:00, 1207.51it/s]

Error processing pair 838844 and 21491791: 'type'
Error processing pair 11658686 and 19434890: 'type'
Error processing pair 1410953 and 20432135: 'type'


Creating PDG pairs: 510it [00:00, 1092.32it/s]

Error processing pair 20100822 and 23677111: 'type'
Error processing pair 87685 and 9210167: 'type'


Creating PDG pairs: 756it [00:00, 1114.33it/s]

...

Epoch 98/100, Train Loss: 0.1186, Val Loss: 0.1160, Val F1: 0.9643


Evaluating: 100%|██████████| 23/23 [00:02<00:00,  9.56it/s]


Epoch 99/100, Train Loss: 0.1201, Val Loss: 0.1160, Val F1: 0.9643


Evaluating: 100%|██████████| 23/23 [00:02<00:00,  8.31it/s]

Epoch 100/100, Train Loss: 0.1214, Val Loss: 0.1160, Val F1: 0.9643


In [15]:
checkpoint = torch.load('checkpoints_100_epoch/checkpoints_CPG/5000_sample/model_epoch_80_f1_0.9643.pth')
per_type_metrics = checkpoint['metrics']['per_type_metrics']

# Print overall metrics first
overall_metrics = per_type_metrics['overall']
print("Overall metrics:")
print(f"Accuracy-score: {overall_metrics['accuracy']:.4f}")
print(f"Precision-score: {overall_metrics['precision']:.4f}")
print(f"Recall-score: {overall_metrics['recall']:.4f}")
print(f"F1-score: {overall_metrics['f1']:.4f}")

# Print per-type metrics
for clone_type, metrics in per_type_metrics.items():
    if clone_type != 'overall':  # Skip overall metrics since already printed
        print(f"\n{clone_type}:")
        print(f"Accuracy-score: {metrics['accuracy']:.4f}")
        print(f"Precision-score: {metrics['precision']:.4f}")
        print(f"Recall-score: {metrics['recall']:.4f}")
        print(f"F1-score: {metrics['f1']:.4f}")



Overall metrics:
Accuracy-score: 0.9654
Precision-score: 0.9678
Recall-score: 0.9654
F1-score: 0.9643

Type-1:
Accuracy-score: 1.0000
Precision-score: 1.0000
Recall-score: 1.0000
F1-score: 1.0000

Type-2:
Accuracy-score: 0.9661
Precision-score: 1.0000
Recall-score: 0.8093
F1-score: 0.8946

Type-3:
Accuracy-score: 0.9654
Precision-score: 0.9312
Recall-score: 1.0000
F1-score: 0.9644

Type-4:
Accuracy-score: 0.9993
Precision-score: 1.0000
Recall-score: 0.9980
F1-score: 0.9990
